In [1]:
from torchrecsys.models.retrieval import DeepRetriever
from torchrecsys.external_datasets import Movielens_1M
from torchrecsys.datasets import InteractionsDataset
from pytorch_lightning import Trainer
import pandas as pd
import torch
import numpy as np
from torch.utils.data import DataLoader

In [2]:
data = Movielens_1M()
ratings, users, movies = data.load()

In [3]:
#Preprocess users
users['gender'], uniques = pd.factorize(users['gender'])
users['occupation'], uniques = pd.factorize(users['occupation'])
users['zip'], uniques = pd.factorize(users['zip'])
##Set category dtype
users['gender'] = users.gender.astype('category')
users['occupation'] = users.occupation.astype('category')
users['zip'] = users.zip.astype('category')

#Preprocess movies, 
##categories to index
movies['title'], uniques = pd.factorize(movies['title'])
movies['genres'], uniques = pd.factorize(movies['genres'])
##Set category dtype
movies['title'] = movies.title.astype('category')
movies['genres'] = movies.genres.astype('category')

ratings["rating"] = 1 #Make all ratings an implicit interaction

In [4]:
dataset = InteractionsDataset(ratings, users, movies, item_id="movie_id", interaction_id="rating", sample_negatives=3)
train = DataLoader(dataset, batch_size=2048)


In [5]:
model = DeepRetriever(dataset.data_schema)

In [6]:
trainer = Trainer(max_epochs=1)
trainer.fit(model, train)

/home/jiwidi/miniconda3/envs/cuda/lib/python3.8/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/jiwidi/miniconda3/envs/cuda/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:118: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name           | Type              | Params
-------------------------------------

Training: 0it [00:00, ?it/s]

/home/jiwidi/miniconda3/envs/cuda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [10]:
candidates = model.generate_item_representations(dataset)

In [11]:
candidates.shape

torch.Size([3883, 256])

In [12]:
from torchrecsys.layers import retrieve_nearest_neighbors

In [13]:
retrieve_nearest_neighbors(candidates, candidates[1]) # WORKJS HEHEHEHE WII

tensor([   1, 1857, 1270])